# Avaliação DESAFIADORA: Modelo Padrão vs ISR REAL

## Sextant Banking Edition - Teste de Robustez com Métricas Completas

**Autor:** SK-Crossroads  
**Data:** Janeiro 2026  
**Versão:** 5.2 (Casos Desafiadores + ISR REAL)

---

### O que esta versão testa

| Tipo de Armadilha | Exemplo | Decisão Esperada |
|-------------------|---------|------------------|
| Score alto + default recente | Score 820, default há 1 mês | NEGADA |
| Score bom + endividamento oculto | Score 750, endiv. 92% | NEGADA |
| Score baixo + cliente excelente | Score 580, 25 anos sem problemas | ANALISE_GERENCIAL |
| PEP não declarado | Secretária de Estado, pep=false | ANALISE_GERENCIAL |
| Renda incompatível | Estagiário com 45k/mês | ANALISE_GERENCIAL |
| Cliente sancionado | Score perfeito mas sancionado=true | NEGADA |
| Menor de idade | Nascido em 2010 | NEGADA |
| Saldo negativo | Score bom, saldo=-5000 | NEGADA |

### Comparação

- **Modelo Padrão**: Apenas LLM com políticas do banco
- **Modelo + ISR REAL**: LLM + `src/tools/isr_auditor.py` (Information Sufficiency Ratio)

### ISR Real - 4 Patches Implementados

1. **Laplace Smoothing** (PROB_FLOOR): Evita divisão por zero
2. **One-Sided Clipping**: Foco na perda de informação
3. **Hard Veto**: Detecta instabilidade severa
4. **Success Shortcut**: Bypass para alta confiança

---
## 1. Setup

In [ ]:
import os
import sys
import json
import math
import numpy as np
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any
from datetime import datetime, date
from copy import deepcopy
from dotenv import load_dotenv

def find_project_root():
    current = Path.cwd()
    if current.name == "notebooks":
        return current.parent
    for parent in [current] + list(current.parents):
        if (parent / "data").exists():
            return parent
    return Path("/home/dumoura/Kunumi/Hallucinations_ISR_V4")

PROJECT_ROOT = find_project_root()
sys.path.insert(0, str(PROJECT_ROOT))
load_dotenv(PROJECT_ROOT / ".env")

print(f"[OK] Projeto: {PROJECT_ROOT}")
print(f"[INFO] Versão 5.2: CASOS DESAFIADORES + ISR REAL")
print(f"[INFO] ISR usando implementação de src/tools/isr_auditor.py")

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY não encontrada!")

client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o-mini"

print(f"[OK] Modelo: {MODEL_NAME}")

---
## 2. Carregando Dados

In [ ]:
# Carregar políticas e casos desafiadores
with open(PROJECT_ROOT / "data" / "raw" / "banco_politicas_diretrizes.md", "r", encoding="utf-8") as f:
    POLITICAS_BANCO = f.read()

with open(PROJECT_ROOT / "data" / "test" / "clientes_teste_desafiadores.json", "r", encoding="utf-8") as f:
    dados_desafiadores = json.load(f)

clientes_desafiadores = dados_desafiadores["clientes"]
print(f"[OK] {len(clientes_desafiadores)} casos desafiadores carregados")

# Distribuição
decisoes = Counter(c.get("_ground_truth", {}).get("decisao_esperada", "N/A") for c in clientes_desafiadores)
print(f"\nDistribuição de decisões esperadas:")
for dec, count in sorted(decisoes.items()):
    print(f"  - {dec}: {count}")

In [ ]:
# Preparar dataset
def preparar_cliente_para_modelo(cliente: Dict) -> Dict:
    return {k: v for k, v in cliente.items() if not k.startswith("_")}

def extrair_ground_truth(cliente: Dict) -> Dict:
    gt = cliente.get("_ground_truth", {})
    return {
        "decisao_esperada": gt.get("decisao_esperada", "NEGADA"),
        "motivo": gt.get("motivo", "Não especificado"),
        "tipo_teste": gt.get("tipo_teste", "unknown")
    }

dataset = []
for cliente in clientes_desafiadores:
    dataset.append({
        "cliente_para_modelo": preparar_cliente_para_modelo(cliente),
        "cliente_original": cliente,
        "ground_truth": extrair_ground_truth(cliente),
        "cliente_id": cliente["cliente_id"]
    })

print(f"[OK] Dataset preparado: {len(dataset)} casos")

---
## 3. Funções do Modelo Padrão

In [ ]:
def criar_system_prompt(politicas: str) -> str:
    return f"""Você é um analista de crédito do Banco S.A.

Analise solicitações de crédito seguindo as políticas do banco.

ATENÇÃO ESPECIAL:
- Verifique TODOS os campos, não apenas o score

---

# POLÍTICAS DO BANCO

{politicas[:18000]}
"""

def criar_prompt_analise(cliente: Dict) -> str:
    cliente_json = json.dumps(cliente, indent=2, ensure_ascii=False, default=str)
    return f"""# ANÁLISE DE CRÉDITO

## Dados do Cliente
```json
{cliente_json}
```

## Instruções
Analise TODOS os campos. Responda em JSON:
{{
  "decisao": "APROVADA" | "NEGADA" | "ANALISE_GERENCIAL",
  "justificativa": "<explicação>",
  "alertas": ["<pontos de atenção>"]
}}
"""

def extrair_json(texto: str) -> Dict:
    if "```json" in texto:
        inicio = texto.find("```json") + 7
        fim = texto.find("```", inicio)
        if fim > inicio:
            try:
                return json.loads(texto[inicio:fim].strip())
            except:
                pass
    inicio = texto.find("{")
    fim = texto.rfind("}")
    if inicio >= 0 and fim > inicio:
        try:
            return json.loads(texto[inicio:fim+1])
        except:
            pass
    return {"erro": "JSON não encontrado"}

def chamar_modelo_padrao(cliente: Dict, system_prompt: str) -> Dict:
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": criar_prompt_analise(cliente)}
            ],
            max_tokens=1024,
            temperature=0.3
        )
        texto = response.choices[0].message.content
        return {"sucesso": True, "resposta_json": extrair_json(texto)}
    except Exception as e:
        return {"sucesso": False, "erro": str(e)}

print("[OK] Funções do modelo padrão definidas")

---
## 4. Modelo com ISR Real (Information Sufficiency Ratio)

O ISR usa a implementação **real** de `src/tools/isr_auditor.py` com:

1. **Laplace Smoothing** (PROB_FLOOR) - evita divisão por zero
2. **One-Sided Clipping** - foco na perda de informação
3. **Hard Veto** - detecta instabilidade severa
4. **Success Shortcut** - bypass para alta confiança

O processo:
1. Modelo padrão gera uma decisão
2. ISR verifica a consistência da decisão via permutações
3. Se ISR detecta instabilidade → bloqueia a decisão

In [ ]:
# Importar a implementação REAL do ISR
from src.tools.isr_auditor import SemanticISRAuditorTool

# Criar instância do auditor ISR real
isr_auditor = SemanticISRAuditorTool(
    client=client,
    model=MODEL_NAME,
    target_confidence=0.95,
    num_permutations=6,
    clipping_b=12.0,
    hard_veto_threshold=0.20
)

print(f"[OK] ISR Auditor Real carregado")
print(f"    - Modelo: {MODEL_NAME}")
print(f"    - Permutações: {isr_auditor.num_permutations}")
print(f"    - Target Confidence: {isr_auditor.target_confidence}")
print(f"    - Hard Veto Threshold: {isr_auditor.hard_veto_threshold}")


def chamar_modelo_com_isr_real(cliente: Dict, system_prompt: str) -> Dict:
    """
    Chama o modelo padrão e depois usa o ISR REAL para verificar a decisão.
    
    O ISR analisa a consistência da decisão via permutações de contexto.
    Se detectar instabilidade severa, bloqueia a decisão.
    """
    # Passo 1: Chamar modelo padrão
    resp_modelo = chamar_modelo_padrao(cliente, system_prompt)
    
    if not resp_modelo["sucesso"]:
        return resp_modelo
    
    decisao_modelo = resp_modelo["resposta_json"].get("decisao", "ERRO")
    justificativa_modelo = resp_modelo["resposta_json"].get("justificativa", "")
    
    # Passo 2: Construir contexto para o ISR
    cliente_json = json.dumps(cliente, indent=2, ensure_ascii=False, default=str)
    prompt_context = f"""POLÍTICAS DO BANCO:
{system_prompt[:8000]}

DADOS DO CLIENTE:
{cliente_json}

ANÁLISE DO MODELO:
Decisão: {decisao_modelo}
Justificativa: {justificativa_modelo}
"""
    
    # Passo 3: Usar ISR REAL para verificar a decisão
    try:
        isr_result_json = isr_auditor.audit(
            prompt_context=prompt_context,
            proposed_decision=decisao_modelo
        )
        isr_result = json.loads(isr_result_json)
        
        isr_decision = isr_result.get("decision", "ERRO")
        isr_metrics = isr_result.get("metrics", {})
        isr_reason = isr_result.get("reason", "")
        
        # Passo 4: Aplicar decisão do ISR
        if isr_decision == "BLOQUEADO":
            # ISR detectou instabilidade - sobrescrever decisão
            # Se o modelo aprovou mas ISR bloqueou → forçar análise gerencial
            if decisao_modelo == "APROVADA":
                decisao_final = "ANALISE_GERENCIAL"
                justificativa_final = f"ISR BLOQUEOU: {isr_reason}. Decisão original: {decisao_modelo}"
            else:
                # Se já era negativa, manter
                decisao_final = decisao_modelo
                justificativa_final = f"{justificativa_modelo} [ISR confirmou rejeição]"
            
            return {
                "sucesso": True,
                "resposta_json": {
                    "decisao": decisao_final,
                    "justificativa": justificativa_final,
                    "alertas": resp_modelo["resposta_json"].get("alertas", []) + ["ISR_BLOQUEIO"]
                },
                "isr_aplicado": True,
                "isr_acao": "BLOQUEIO",
                "isr_metrics": isr_metrics,
                "isr_reason": isr_reason
            }
        else:
            # ISR aprovou - manter decisão original
            return {
                "sucesso": True,
                "resposta_json": resp_modelo["resposta_json"],
                "isr_aplicado": True,
                "isr_acao": "APROVADO",
                "isr_metrics": isr_metrics,
                "isr_reason": isr_reason
            }
            
    except Exception as e:
        print(f"    [ERRO ISR] {e}")
        # Em caso de erro, retornar resultado do modelo sem ISR
        return {
            "sucesso": True,
            "resposta_json": resp_modelo["resposta_json"],
            "isr_aplicado": False,
            "isr_acao": "ERRO",
            "isr_metrics": {},
            "isr_reason": str(e)
        }

print("[OK] Função chamar_modelo_com_isr_real definida")

---
## 5. Executando Testes Comparativos

In [ ]:
import time

SYSTEM_PROMPT = criar_system_prompt(POLITICAS_BANCO)

resultados_padrao = []
resultados_isr = []

print("Executando testes comparativos com ISR REAL...")
print("="*90)
print("NOTA: ISR Real usa permutações de contexto - mais lento mas mais preciso")
print("="*90)

for i, item in enumerate(dataset):
    cliente = item["cliente_para_modelo"]
    gt = item["ground_truth"]
    
    print(f"\n[{i+1}/{len(dataset)}] {item['cliente_id']}")
    print(f"    Esperado: {gt['decisao_esperada']} | Tipo: {gt['tipo_teste']}")
    
    # Modelo Padrão
    print(f"    Padrão: ", end="")
    resp_padrao = chamar_modelo_padrao(cliente, SYSTEM_PROMPT)
    if resp_padrao["sucesso"]:
        dec_padrao = resp_padrao["resposta_json"].get("decisao", "ERRO")
        acertou_padrao = dec_padrao == gt["decisao_esperada"]
        print(f"{dec_padrao} {'✓' if acertou_padrao else '✗'}")
    else:
        dec_padrao = "ERRO"
        acertou_padrao = False
        print(f"ERRO")
    
    resultados_padrao.append({
        "cliente_id": item["cliente_id"],
        "tipo_teste": gt["tipo_teste"],
        "esperado": gt["decisao_esperada"],
        "obtido": dec_padrao,
        "acertou": acertou_padrao
    })
    
    time.sleep(0.3)
    
    # Modelo com ISR REAL
    print(f"    ISR:    ", end="")
    resp_isr = chamar_modelo_com_isr_real(cliente, SYSTEM_PROMPT)
    if resp_isr["sucesso"]:
        dec_isr = resp_isr["resposta_json"].get("decisao", "ERRO")
        acertou_isr = dec_isr == gt["decisao_esperada"]
        isr_acao = resp_isr.get("isr_acao", "N/A")
        isr_metrics = resp_isr.get("isr_metrics", {})
        isr_value = isr_metrics.get("ISR", "N/A")
        print(f"{dec_isr} {'✓' if acertou_isr else '✗'} (ISR: {isr_acao}, ISR={isr_value})")
    else:
        dec_isr = "ERRO"
        acertou_isr = False
        isr_acao = "ERRO"
        isr_metrics = {}
        print(f"ERRO")
    
    resultados_isr.append({
        "cliente_id": item["cliente_id"],
        "tipo_teste": gt["tipo_teste"],
        "esperado": gt["decisao_esperada"],
        "obtido": dec_isr,
        "acertou": acertou_isr,
        "isr_acao": isr_acao,
        "isr_metrics": isr_metrics,
        "isr_reason": resp_isr.get("isr_reason", "")
    })
    
    time.sleep(0.5)

print("\n" + "="*90)
print("[OK] Testes concluídos!")

---
## 6. Calculando Métricas de ML

In [ ]:
def calcular_metricas_ml(resultados: List[Dict]) -> Dict:
    """
    Calcula métricas de ML para classificação multiclasse.
    
    Para métricas binárias (precision, recall, F1):
    - POSITIVO = APROVADA (conceder crédito)
    - NEGATIVO = NEGADA ou ANALISE_GERENCIAL (não conceder automaticamente)
    """
    valid = [r for r in resultados if r["obtido"] != "ERRO"]
    if not valid:
        return {"erro": "Sem resultados válidos"}
    
    total = len(valid)
    acertos = sum(1 for r in valid if r["acertou"])
    
    # Accuracy geral
    accuracy = acertos / total
    
    # Métricas binárias (APROVADA vs resto)
    def to_binary(dec):
        return 1 if dec == "APROVADA" else 0
    
    y_true = [to_binary(r["esperado"]) for r in valid]
    y_pred = [to_binary(r["obtido"]) for r in valid]
    
    tp = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 1)
    tn = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 0)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)  # Aprovou indevidamente
    fn = sum(1 for t, p in zip(y_true, y_pred) if t == 1 and p == 0)  # Negou indevidamente
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Matriz de confusão multiclasse
    classes = ["APROVADA", "NEGADA", "ANALISE_GERENCIAL"]
    matriz = {}
    for esp in classes:
        matriz[esp] = {obt: 0 for obt in classes}
    
    for r in valid:
        esp = r["esperado"]
        obt = r["obtido"]
        if esp in matriz and obt in matriz[esp]:
            matriz[esp][obt] += 1
    
    return {
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "confusion_matrix_binary": {"TP": tp, "TN": tn, "FP": fp, "FN": fn},
        "confusion_matrix_multi": matriz,
        "total": total,
        "acertos": acertos,
        "false_positives": fp  # Aprovações indevidas (risco!)
    }

metricas_padrao = calcular_metricas_ml(resultados_padrao)
metricas_isr = calcular_metricas_ml(resultados_isr)

print("[OK] Métricas calculadas!")

---
## 7. Resultados: Modelo Padrão

In [ ]:
print("="*70)
print("MODELO PADRÃO (LLM sem validação adicional)")
print("="*70)

if "erro" not in metricas_padrao:
    print(f"\nAcertos: {metricas_padrao['acertos']}/{metricas_padrao['total']}")
    
    print(f"\nMétricas:")
    print(f"  Accuracy:  {metricas_padrao['accuracy']:.1%}")
    print(f"  Precision: {metricas_padrao['precision']:.1%}")
    print(f"  Recall:    {metricas_padrao['recall']:.1%}")
    print(f"  F1-Score:  {metricas_padrao['f1_score']:.1%}")
    
    cm = metricas_padrao["confusion_matrix_binary"]
    print(f"\nMatriz de Confusão (Binária):")
    print(f"  TP (Aprovou certo):    {cm['TP']}")
    print(f"  TN (Negou certo):      {cm['TN']}")
    print(f"  FP (RISCO - aprovação indevida): {cm['FP']}")
    print(f"  FN (Negou indevidamente): {cm['FN']}")
else:
    print(f"ERRO: {metricas_padrao['erro']}")

---
## 8. Resultados: Modelo com ISR REAL

**ISR Real** (Information Sufficiency Ratio) da implementação `src/tools/isr_auditor.py`:
- Verifica consistência via permutações de contexto
- Calcula métricas: Delta, B2T, JS_Bound
- Aplica Hard Veto para instabilidade severa

In [ ]:
print("="*70)
print("MODELO COM ISR REAL (implementação de src/tools/isr_auditor.py)")
print("="*70)

if "erro" not in metricas_isr:
    print(f"\nAcertos: {metricas_isr['acertos']}/{metricas_isr['total']}")
    
    print(f"\nMétricas de Classificação:")
    print(f"  Accuracy:  {metricas_isr['accuracy']:.1%}")
    print(f"  Precision: {metricas_isr['precision']:.1%}")
    print(f"  Recall:    {metricas_isr['recall']:.1%}")
    print(f"  F1-Score:  {metricas_isr['f1_score']:.1%}")
    
    cm = metricas_isr["confusion_matrix_binary"]
    print(f"\nMatriz de Confusão (Binária):")
    print(f"  TP (Aprovou certo):    {cm['TP']}")
    print(f"  TN (Negou certo):      {cm['TN']}")
    print(f"  FP (RISCO - aprovação indevida): {cm['FP']}")
    print(f"  FN (Negou indevidamente): {cm['FN']}")
    
    # Mostrar ações do ISR
    acoes_isr = Counter(r.get("isr_acao", "N/A") for r in resultados_isr)
    print(f"\nAções do ISR Real:")
    for acao, count in sorted(acoes_isr.items()):
        print(f"  - {acao}: {count}")
    
    # Mostrar estatísticas das métricas ISR
    isr_values = [r.get("isr_metrics", {}).get("ISR", None) for r in resultados_isr]
    isr_values = [v for v in isr_values if v is not None and v != 999.0]  # Filtrar valores válidos
    
    if isr_values:
        print(f"\nEstatísticas do ISR (Information Sufficiency Ratio):")
        print(f"  ISR Médio: {np.mean(isr_values):.4f}")
        print(f"  ISR Mínimo: {np.min(isr_values):.4f}")
        print(f"  ISR Máximo: {np.max(isr_values):.4f}")
        print(f"  ISR >= 1.0 (aprovados): {sum(1 for v in isr_values if v >= 1.0)}")
        print(f"  ISR < 1.0 (bloqueados): {sum(1 for v in isr_values if v < 1.0)}")
else:
    print(f"ERRO: {metricas_isr['erro']}")

---
## 9. Comparação Final

In [ ]:
print("="*80)
print("COMPARAÇÃO: MODELO PADRÃO vs MODELO COM ISR")
print("="*80)

if "erro" not in metricas_padrao and "erro" not in metricas_isr:
    print(f"\n{'Métrica':<25} {'Modelo Padrão':>15} {'Modelo + ISR':>15} {'Diferença':>15}")
    print("-"*80)
    
    # Accuracy
    diff = metricas_isr['accuracy'] - metricas_padrao['accuracy']
    print(f"{'Accuracy':<25} {metricas_padrao['accuracy']:>14.1%} {metricas_isr['accuracy']:>14.1%} {diff:>+14.1%}")
    
    # Precision
    diff = metricas_isr['precision'] - metricas_padrao['precision']
    print(f"{'Precision':<25} {metricas_padrao['precision']:>14.1%} {metricas_isr['precision']:>14.1%} {diff:>+14.1%}")
    
    # Recall
    diff = metricas_isr['recall'] - metricas_padrao['recall']
    print(f"{'Recall':<25} {metricas_padrao['recall']:>14.1%} {metricas_isr['recall']:>14.1%} {diff:>+14.1%}")
    
    # F1-Score
    diff = metricas_isr['f1_score'] - metricas_padrao['f1_score']
    print(f"{'F1-Score':<25} {metricas_padrao['f1_score']:>14.1%} {metricas_isr['f1_score']:>14.1%} {diff:>+14.1%}")
    
    print("-"*80)
    
    # False Positives (CRÍTICO)
    diff = metricas_isr['false_positives'] - metricas_padrao['false_positives']
    print(f"{'False Positives (RISCO)':<25} {metricas_padrao['false_positives']:>15} {metricas_isr['false_positives']:>15} {diff:>+15}")
    
    print("="*80)
    
    # Análise
    melhoria_accuracy = (metricas_isr['accuracy'] - metricas_padrao['accuracy']) / metricas_padrao['accuracy'] * 100 if metricas_padrao['accuracy'] > 0 else 0
    reducao_fp = metricas_padrao['false_positives'] - metricas_isr['false_positives']
    
    print(f"\n[ANÁLISE]")
    print(f"  Melhoria na Accuracy: {melhoria_accuracy:+.1f}%")
    print(f"  Redução de False Positives: {reducao_fp} casos")
    
    if metricas_isr['accuracy'] > metricas_padrao['accuracy']:
        print(f"\n  [CONCLUSÃO] ISR MELHOROU o desempenho do modelo!")
    elif metricas_isr['accuracy'] == metricas_padrao['accuracy']:
        print(f"\n  [CONCLUSÃO] ISR manteve o desempenho (sem impacto negativo)")
    else:
        print(f"\n  [CONCLUSÃO] ISR reduziu accuracy (mas pode ter reduzido risco)")

---
## 10. Análise por Tipo de Armadilha

In [ ]:
print("="*90)
print("ANÁLISE POR TIPO DE ARMADILHA")
print("="*90)

print(f"\n{'Tipo de Teste':<45} {'Padrão':>12} {'ISR':>12} {'Melhoria':>12}")
print("-"*90)

tipos = set(r["tipo_teste"] for r in resultados_padrao)

for tipo in sorted(tipos):
    # Padrão
    casos_padrao = [r for r in resultados_padrao if r["tipo_teste"] == tipo]
    acertos_padrao = sum(1 for r in casos_padrao if r["acertou"])
    total_tipo = len(casos_padrao)
    taxa_padrao = acertos_padrao / total_tipo if total_tipo > 0 else 0
    
    # ISR
    casos_isr = [r for r in resultados_isr if r["tipo_teste"] == tipo]
    acertos_isr = sum(1 for r in casos_isr if r["acertou"])
    taxa_isr = acertos_isr / total_tipo if total_tipo > 0 else 0
    
    # Melhoria
    melhoria = "✓" if taxa_isr > taxa_padrao else ("=" if taxa_isr == taxa_padrao else "✗")
    
    print(f"{tipo:<45} {taxa_padrao:>11.0%} {taxa_isr:>11.0%} {melhoria:>12}")

---
## 11. Salvando Resultados

In [ ]:
output_dir = PROJECT_ROOT / "outputs" / "metrics"
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"comparacao_desafiador_isr_real_v5_{timestamp}.json"

# Limpar resultados para JSON
def limpar_para_json(obj):
    """Converte tipos numpy e bool para tipos nativos Python."""
    if isinstance(obj, dict):
        return {k: limpar_para_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [limpar_para_json(v) for v in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    else:
        return obj

def limpar_resultados(resultados):
    limpos = []
    for r in resultados:
        item = {
            "cliente_id": r["cliente_id"],
            "tipo_teste": r["tipo_teste"],
            "esperado": r["esperado"],
            "obtido": r["obtido"],
            "acertou": bool(r["acertou"])
        }
        # Adicionar métricas ISR se existirem
        if "isr_acao" in r:
            item["isr_acao"] = r["isr_acao"]
        if "isr_metrics" in r:
            item["isr_metrics"] = limpar_para_json(r["isr_metrics"])
        if "isr_reason" in r:
            item["isr_reason"] = r["isr_reason"]
        limpos.append(item)
    return limpos

dados_salvar = {
    "metadata": {
        "timestamp": timestamp,
        "modelo": MODEL_NAME,
        "versao": "5.2-desafiador-isr-real",
        "total_casos": len(dataset),
        "isr_config": {
            "target_confidence": isr_auditor.target_confidence,
            "num_permutations": isr_auditor.num_permutations,
            "clipping_b": isr_auditor.clipping_b,
            "hard_veto_threshold": isr_auditor.hard_veto_threshold,
            "prob_floor": isr_auditor.PROB_FLOOR
        }
    },
    "metricas_padrao": limpar_para_json(metricas_padrao),
    "metricas_isr": limpar_para_json(metricas_isr),
    "resultados_padrao": limpar_resultados(resultados_padrao),
    "resultados_isr": limpar_resultados(resultados_isr)
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dados_salvar, f, indent=2, ensure_ascii=False, default=str)

print(f"[OK] Resultados salvos: {output_file}")

---
## 12. Conclusões

### ISR Real - Como funciona

O ISR (Information Sufficiency Ratio) verifica se o modelo tem informação suficiente para a decisão:

| Métrica ISR | Significado | Ação |
|-------------|-------------|------|
| **ISR >= 1.0** | Informação suficiente | APROVADO |
| **ISR < 1.0** | Informação insuficiente | BLOQUEADO |
| **ISR = 999** | Alta confiança (shortcut) | APROVADO imediato |
| **Hard Veto** | Instabilidade severa | BLOQUEADO imediato |

### Métricas de Classificação

| Métrica | O que mede | Importância para Crédito |
|---------|------------|-------------------------|
| **Accuracy** | % de acertos totais | Visão geral |
| **Precision** | Dos aprovados, quantos eram corretos | Evita aprovar indevidamente |
| **Recall** | Dos que deveriam aprovar, quantos aprovou | Evita perder bons clientes |
| **F1-Score** | Equilíbrio precision/recall | Métrica balanceada |
| **False Positives** | Aprovações indevidas | **CRÍTICO para risco** |

### Interpretação do ISR Real

O ISR Real analisa a **consistência** da decisão do modelo:
- Se o modelo é inconsistente entre permutações → ISR baixo → Bloqueio
- Se o modelo é consistente e confiante → ISR alto → Aprovação

### Diferença do ISR Real vs Regras Hard-coded

| Aspecto | Regras Hard-coded | ISR Real |
|---------|-------------------|----------|
| **Verificação** | Lista fixa de regras | Análise de consistência |
| **Adaptabilidade** | Rígido | Detecta padrões implícitos |
| **Cobertura** | Apenas casos previstos | Qualquer caso com instabilidade |
| **Custo** | Baixo (sem API) | Alto (múltiplas chamadas) |

### Recomendações

1. **ISR Real** é ideal para detectar incerteza do modelo, não apenas casos específicos
2. O modelo pode "acertar" mas com baixa confiança - ISR captura isso
3. Combine ISR com regras de negócio para cobertura completa